In [1]:
!rm -f aclImdb_v1.tar.gz
!rm -Rf aclImdb
!rm -f movie_data.csv
!rm -f movie_data_train.csv
!rm -f movie_data_validation.csv

In [2]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2018-09-27 20:57:44--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  9.22MB/s    in 11s     

2018-09-27 20:57:55 (7.54 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [3]:
!tar xzf aclImdb_v1.tar.gz

In [4]:
import os

numberOfFiles = 0
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = './aclImdb/%s/%s' % (s, l)
        numberOfFiles += len(os.listdir(path))

print 'Number of files in dataset ' + str(numberOfFiles)

Number of files in dataset 50000


In [5]:
!pip install pyprind
!pip install nltk

distributed 1.21.8 requires msgpack, which is not installed.
grin 1.2.1 requires argparse>=1.1, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
distributed 1.21.8 requires msgpack, which is not installed.
grin 1.2.1 requires argparse>=1.1, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import pyprind
import pandas as pd
import os

pbar = pyprind.ProgBar(numberOfFiles)

labels = {'pos':'1', 'neg':'0'}


df = pd.DataFrame()
for s in ('test', 'train'):
	for l in ('pos', 'neg'):
		path ='./aclImdb/%s/%s' % (s, l)
		
		for file in os.listdir(path):
			with open(os.path.join(path, file), 'r') as infile:
				txt = infile.read()
				df = df.append([[labels[l], txt]], ignore_index=True)
			pbar.update()

df.columns = ['label', 'text']


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:29


In [7]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index=False)

In [8]:
df = pd.read_csv('./movie_data.csv')
df_row_count = len(df.index)
df.head(5)

,label,text
0,1,"Renowned Czech actor Vlastimil Brodský, mostly..."
1,0,"First of all, Katherine Hepburn is badly misca..."
2,0,Absolutely one of the worst movies of all time...
3,1,This movie is a perfect portrayal of The Nutcr...
4,0,I'm sure that Operations Dames was a favorite ...


In [9]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

porter = PorterStemmer()
nltk.download('stopwords')
stop = stopwords.words('english')

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text + ''.join(emoticons).replace('-', '')
    text = tokenizer_porter(text)
    text = [w for w in text if w not in stop]
    text = ' '.join(text)
    return text

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
pbar = pyprind.ProgBar(df_row_count)

for index, row in df.iterrows():
    df.at[index, 'text'] = preprocessor(row['text'])
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:07:20


In [11]:
df.head(5)

,label,text
0,1,renown czech actor vlastimil brodsk mostli kno...
1,0,first katherin hepburn badli miscast clara con...
2,0,absolut one worst movi time low product valu t...
3,1,thi movi perfect portray nutcrack danc wonder ...
4,0,sure oper dame wa favorit drive back day absol...


In [12]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(df, test_size=0.2)

In [14]:
import csv

!rm -Rf data
!mkdir data

train.to_csv('data/train', header = False, sep='\t', index=False, quoting=csv.QUOTE_NONE)
validation.to_csv('data/test', header = False, sep='\t', index=False, quoting=csv.QUOTE_NONE)

In [30]:
pd.read_csv('data/train', header = None, sep='\t').head(5)

,0,1
0,1,thi film far enjoy rate 7 would suggest mani w...
1,1,oh dear like super hero love interest develop ...
2,1,purchas restor version film rememb much affect...
3,1,manhattan apart dweller put kind inconveni wor...
4,1,found thi film one great heart warm gem stori ...


In [31]:
pd.read_csv('data/test', header = None, sep='\t').head(5)

,0,1
0,1,everi onc wonder world horror diamond craft on...
1,0,thi must one worst movi ever seen disagre anot...
2,0,would lie rel wa crew member got go watch prod...
3,1,fulci one time favorit italian splatter direct...
4,1,black water ha one best australian movi seen m...
